## Abstract

Despite the header being `## Abstract`, this section will render as a highlighted section titled *Summary*.

----

:::{.callout-note title="AI usage disclosure" collapse="true"}
This is a placeholder for the AI usage disclosure. Once all authors sign the AI code form on AirTable, SlackBot will message you an AI disclosure that you should place here.
:::

## Introduction

Once edited by you, this file will become your publication. Alternatively, if you already have a notebook written that you're trying to transform into a pub, replace this file with your own, but be sure to add the YAML front matter (the first cell) to your notebook.

## Learn more

Refer to [Demo](/examples/demo.html) to learn about possible syntax.